In [11]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

In [12]:
print(torch.__version__)

1.8.1+cu101


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, 3)
    self.conv2 = nn.Conv2d(6,16, 3)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, int(x.nelement() / x.shape[0]))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    return x

In [15]:
model = LeNet().to(device=device)

In [16]:
module = model.conv1
print(list(module.named_parameters()))

[('weight', Parameter containing:
tensor([[[[ 0.0599, -0.1869, -0.0625],
          [-0.2979, -0.2901,  0.0591],
          [ 0.0817, -0.0567,  0.0737]]],


        [[[ 0.1089, -0.2292, -0.1351],
          [ 0.3329, -0.0468,  0.2500],
          [ 0.2478, -0.0239,  0.2210]]],


        [[[ 0.2232, -0.2996,  0.2256],
          [-0.0457,  0.3121, -0.2596],
          [ 0.1272, -0.3175, -0.2620]]],


        [[[-0.0087, -0.3141, -0.2709],
          [-0.1339,  0.2129,  0.2787],
          [ 0.3055, -0.0027, -0.1207]]],


        [[[-0.1806,  0.2542,  0.1914],
          [-0.1549,  0.2211, -0.3105],
          [ 0.2290,  0.2645, -0.2129]]],


        [[[-0.0682,  0.2135,  0.1781],
          [ 0.2427, -0.3202,  0.2403],
          [-0.1330, -0.1835,  0.1608]]]], device='cuda:0', requires_grad=True)), ('bias', Parameter containing:
tensor([-0.0558, -0.3073, -0.1109,  0.0847, -0.0711,  0.1325], device='cuda:0',
       requires_grad=True))]


In [17]:
print(list(module.named_buffers()))

[]


In [18]:
prune.random_unstructured(module, name="weight", amount=0.3)

Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))

모듈에 대해 가지치기 기법을 적용하기 위해 첫 번째로는, torch.nn.utils.prune 내 존재하는 가지치기 기법을 선택합니다.   
  
그 후, 해당 모듈 내에서 가지치기 기법을 적용하고자 하는 모듈과 파라미터를 지정합니다. 
  
마지막으로, 가지치기 기법에 적당한 키워드 인자값을 이용하여 가지치기 매개변수를 지정합니다.   

이번 예제에서는, conv1 층의 가중치의 30%값들을 랜덤으로 가지치기 기법을 적용해보겠습니다.   
모듈은 함수에 대한 첫 번째 인자값으로 전달되며, name 은 문자열 식별자를 이용하여 해당 모듈 내 매개변수를 구분합니다.   
그리고, amount 는 가지치기 기법을 적용하기 위한 대상 가중치값들의 백분율 (0과 1사이의 실수값), 혹은 가중치값의 연결의 개수 (음수가 아닌 정수) 를 지정합니다.

In [19]:
print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([-0.0558, -0.3073, -0.1109,  0.0847, -0.0711,  0.1325], device='cuda:0',
       requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[ 0.0599, -0.1869, -0.0625],
          [-0.2979, -0.2901,  0.0591],
          [ 0.0817, -0.0567,  0.0737]]],


        [[[ 0.1089, -0.2292, -0.1351],
          [ 0.3329, -0.0468,  0.2500],
          [ 0.2478, -0.0239,  0.2210]]],


        [[[ 0.2232, -0.2996,  0.2256],
          [-0.0457,  0.3121, -0.2596],
          [ 0.1272, -0.3175, -0.2620]]],


        [[[-0.0087, -0.3141, -0.2709],
          [-0.1339,  0.2129,  0.2787],
          [ 0.3055, -0.0027, -0.1207]]],


        [[[-0.1806,  0.2542,  0.1914],
          [-0.1549,  0.2211, -0.3105],
          [ 0.2290,  0.2645, -0.2129]]],


        [[[-0.0682,  0.2135,  0.1781],
          [ 0.2427, -0.3202,  0.2403],
          [-0.1330, -0.1835,  0.1608]]]], device='cuda:0', requires_grad=True))]


In [20]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[1., 1., 1.],
          [1., 0., 0.],
          [1., 0., 1.]]],


        [[[1., 0., 1.],
          [1., 0., 1.],
          [1., 0., 1.]]],


        [[[1., 0., 1.],
          [1., 1., 1.],
          [1., 0., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 0.],
          [1., 1., 1.]]],


        [[[0., 1., 1.],
          [1., 0., 0.],
          [1., 1., 0.]]],


        [[[0., 1., 1.],
          [1., 1., 1.],
          [0., 1., 0.]]]], device='cuda:0'))]


In [21]:
print(module.weight)

tensor([[[[ 0.0599, -0.1869, -0.0625],
          [-0.2979, -0.0000,  0.0000],
          [ 0.0817, -0.0000,  0.0737]]],


        [[[ 0.1089, -0.0000, -0.1351],
          [ 0.3329, -0.0000,  0.2500],
          [ 0.2478, -0.0000,  0.2210]]],


        [[[ 0.2232, -0.0000,  0.2256],
          [-0.0457,  0.3121, -0.2596],
          [ 0.1272, -0.0000, -0.2620]]],


        [[[-0.0087, -0.3141, -0.2709],
          [-0.1339,  0.2129,  0.0000],
          [ 0.3055, -0.0027, -0.1207]]],


        [[[-0.0000,  0.2542,  0.1914],
          [-0.1549,  0.0000, -0.0000],
          [ 0.2290,  0.2645, -0.0000]]],


        [[[-0.0000,  0.2135,  0.1781],
          [ 0.2427, -0.3202,  0.2403],
          [-0.0000, -0.1835,  0.0000]]]], device='cuda:0',
       grad_fn=<MulBackward0>)


In [23]:
print(module._forward_pre_hooks)

OrderedDict([(0, <torch.nn.utils.prune.RandomUnstructured object at 0x7f28a6983a10>)])


In [24]:
new_model = LeNet()
for name, module in new_model.named_modules():
  if isinstance(module, torch.nn.Conv2d):
    prune.l1_unstructured(module, name='weight', amount=0.2)
  elif isinstance(module, torch.nn.Linear):
    prune.l1_unstructured(module, name='weight', amount=0.4)
  
print(dict(new_model.named_buffers()).keys())

dict_keys(['conv1.weight_mask', 'conv2.weight_mask', 'fc1.weight_mask', 'fc2.weight_mask', 'fc3.weight_mask'])
